In [ ]:
#Step 1: Install Required Libraries
#pip install pandas numpy scikit-learn ,pip install xgboost tensorflow keras

#Step 2: Import Libraries
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

#Models 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [3]:
#Step 3: Load dataset (หามาโลด ดูตารางค่าที่รับให้ตรง)
#age, weight, height, blood_sugar, a1c, systolic_bp, diastolic_bp, mood, gender, and target

df = pd.read_csv('health_dataset.csv')  

In [4]:
#Step 4: Explore the Data (To know structure)
# Display the first few rows of the dataset
print(df.head())

   age  weight  height  blood_sugar  a1c  systolic_bp  diastolic_bp gender  \
0   46      71     182          NaN  9.7        133.0           NaN    ชาย   
1   44      73     165         94.0  NaN        133.0         109.0    ชาย   
2   80      51     184          NaN  6.9          NaN           NaN    ชาย   
3   46      78     176          NaN  6.3          NaN           NaN    ชาย   
4   27      96     174          NaN  NaN          NaN          83.0   หญิง   

       mood  target  
0     เศร้า       0  
1     เศร้า       0  
2      เฉยๆ       1  
3   ความสุข       0  
4  หงุดหงิด       1  


In [5]:
#Step 4: Explore the Data (To know structure)
# Get summary statistics
print(df.describe())

                age        weight        height   blood_sugar          a1c  \
count  20000.000000  20000.000000  20000.000000  10168.000000  9906.000000   
mean      49.040450     72.495150    170.056250    109.801633     7.024490   
std       18.179839     16.170233     11.781669     23.257588     1.739282   
min       18.000000     45.000000    150.000000     70.000000     4.000000   
25%       33.000000     59.000000    160.000000     90.000000     5.500000   
50%       49.000000     72.000000    170.000000    110.000000     7.000000   
75%       65.000000     87.000000    180.000000    130.000000     8.500000   
max       80.000000    100.000000    190.000000    150.000000    10.000000   

       systolic_bp  diastolic_bp      target  
count  9985.000000   9912.000000  20000.0000  
mean    135.135704     89.923023      0.5035  
std      26.386138     17.576149      0.5000  
min      90.000000     60.000000      0.0000  
25%     112.000000     75.000000      0.0000  
50%     135.000

In [6]:
#Step 4: Explore the Data (To know structure)
# Check for missing values
print(df.isnull().sum())

age                 0
weight              0
height              0
blood_sugar      9832
a1c             10094
systolic_bp     10015
diastolic_bp    10088
gender              0
mood                0
target              0
dtype: int64


In [7]:
#Step 5: Data Preprocessing
#(1)Separate Between Numeric and Categorical Column 

numeric_cols = ['age', 'weight', 'height', 'blood_sugar', 'a1c', 'systolic_bp', 'diastolic_bp']
categorical_cols = ['gender', 'mood']

In [8]:
#Step 5: Data Preprocessing
#(2)Separate Between Numeric and Categorical Column 

df['gender'] = df['gender'].map({'ชาย': 0, 'หญิง': 1})
df['mood'] = df['mood'].map({
    'หัวเราะ': 1, 'ความสุข': 2, 'เฉยๆ': 3, 'หงุดหงิด': 4, 'ป่วย': 5, 'เศร้า': 6, 'โกรธ': 7
})

In [9]:
#Step 5: Data Preprocessing
#Feature Engineering 
#(3)To Delete NAN from Height And Weight
df.dropna(subset=['height', 'weight'], inplace=True)  

#(4)To Calculate BMI from Height And Weight
df['BMI'] = df['weight'] / (df['height'] / 100) ** 2

#(5)กำหนดว่าใดเป็นOptionalบ้าง และถ้าอันไหนไม่มีก็แทนด้วย 0
optional_features = ['blood_sugar', 'a1c', 'systolic_bp', 'diastolic_bp' , 'mood']
for col in optional_features:
    df[f'has_{col}'] = df[col].notna().astype(int)  # 1 = Have Data, 0 = Nan

#(6) แทนค่า missing เฉพาะที่จำเป็น
df.fillna(0, inplace=True)  # แค่แทนที่ missing ใน optional features ด้วย 0 (ถ้าไม่มีข้อมูล)

#(7) กำหนด Features (X) และ Target (y)
X = df.drop(columns=['target'])
y = df['target']


In [10]:
#Step 5: Data Preprocessing

#(8)To Save on New File After Preprocessing
df.to_csv('cleaned_health_dataset.csv', index=False)

#(9)Check After Preprocessing 
print(df.head())

   age  weight  height  blood_sugar  a1c  systolic_bp  diastolic_bp  gender  \
0   46      71     182          0.0  9.7        133.0           0.0       0   
1   44      73     165         94.0  0.0        133.0         109.0       0   
2   80      51     184          0.0  6.9          0.0           0.0       0   
3   46      78     176          0.0  6.3          0.0           0.0       0   
4   27      96     174          0.0  0.0          0.0          83.0       1   

   mood  target        BMI  has_blood_sugar  has_a1c  has_systolic_bp  \
0     6       0  21.434609                0        1                1   
1     6       0  26.813590                1        0                1   
2     3       1  15.063800                0        1                0   
3     2       0  25.180785                0        1                0   
4     4       1  31.708284                0        0                0   

   has_diastolic_bp  has_mood  
0                 0         1  
1                 1   

In [11]:
#Step 6: Split the Data into Training and Testing Sets
# Define features and target
X = df.drop(columns=['target'])
y = df['target']

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
#Step 7: Scale the Features For Better For better model performance And Scale The Features using StandardScaler
# Scale the features ANN and XGBoost
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [13]:
#Step 8: Train the Model

#(1)Train a RandomForestClassifier model 
#Hyperparameter Tuning
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=3, n_jobs=-1)
rf_grid.fit(X_train_scaled, y_train)
rf_model = rf_grid.best_estimator_

rf_pred = rf_model.predict(X_test_scaled)
rf_acc = accuracy_score(y_test, rf_pred)
rf_f1 = classification_report(y_test, rf_pred, output_dict=True)['weighted avg']['f1-score']

In [14]:
#Step 8: Train the Model

#(2)Train a XGBClassifie model
#Hyperparameter Tuning
xgb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9]
}
xgb_grid = GridSearchCV(XGBClassifier(eval_metric='logloss', random_state=42), 
                        xgb_params, cv=3, n_jobs=-1)
xgb_grid.fit(X_train_scaled, y_train)
xgb_model = xgb_grid.best_estimator_

xgb_pred = xgb_model.predict(X_test_scaled)
xgb_acc = accuracy_score(y_test, xgb_pred)
xgb_f1 = classification_report(y_test, xgb_pred, output_dict=True)['weighted avg']['f1-score']

In [ ]:
#Step 8: Train the Model

#(3)Train a Artificial Neural Networks (ANNs) model
#Hyperparameter Tuning

ann_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Use sigmoid For Avoid (classification) Problems
])

ann_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
ann_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=0)  # เทรน 50 รอบ
ann_pred = (ann_model.predict(X_test_scaled) > 0.5).astype(int).flatten()
ann_acc = accuracy_score(y_test, ann_pred)
ann_f1 = classification_report(y_test, ann_pred, output_dict=True)['weighted avg']['f1-score']

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [17]:
#Step 9:  Performance
#Show Performance
print(f" Random Forest - Accuracy: {rf_acc*100:.2f}%, F1-score: {rf_f1:.2f}")
print(f" XGBoost - Accuracy: {xgb_acc*100:.2f}%, F1-score: {xgb_f1:.2f}")
print(f" ANN - Accuracy: {ann_acc*100:.2f}%, F1-score: {ann_f1:.2f}")

#Show Best Performance Model
best_model = max([
    (rf_model, rf_acc, rf_f1, "Random Forest"),
    (xgb_model, xgb_acc, xgb_f1, "XGBoost"),
    (ann_model, ann_acc, ann_f1, "ANN")
], key=lambda x: x[2])

print(f" The Best Model is: {best_model[3]} | Accuracy: {best_model[1]*100:.2f}% | F1-score: {best_model[2]:.2f}")

 Random Forest - Accuracy: 50.10%, F1-score: 0.50
 XGBoost - Accuracy: 50.40%, F1-score: 0.50
 ANN - Accuracy: 49.00%, F1-score: 0.49
 The Best Model is: XGBoost | Accuracy: 50.40% | F1-score: 0.50


In [22]:
#Step 10: Make Predictions and Provide Recommendations

# Prediction
def predict_health(input_data):
    health_score = 10  # Start From. . . 10
    risk_percentage = 0  # Risk Percentage
    # BMI
    if input_data['BMI'] > 30:
        health_score -= 2  # minus 2
    elif input_data['BMI'] > 25:
        health_score -= 1  # plus 1

    # Systolic BP
    if input_data['systolic_bp'] > 140:
        health_score -= 2  # minus 2
    elif input_data['systolic_bp'] > 130:
        health_score -= 1  # plus 1

    # Blood sugar
    if input_data['blood_sugar'] > 180:
        health_score -= 3  # minus 3
        risk_percentage += 40  # More Risk 40%
    elif input_data['blood_sugar'] > 120:
        risk_percentage += 20  # More Risk 20%

    # Mood 
    if input_data['mood'] == 'เศร้า':
        health_score -= 1  # minus 1
    elif input_data['mood'] == 'หงุดหงิด':
        health_score -= 1  # minus1



    return health_score, risk_percentage 

# Recommendation
def health_recommendations(input_data):
    recommendations = {}

    # Exercise Recommendation
    if input_data['BMI'] > 30:
        exercise_recommendation = "การออกกำลังกาย: แนะนำให้เดินเร็ว 30 นาทีต่อวัน หรือทำโยคะเพื่อเพิ่มความยืดหยุ่น. ควรทำอย่างสม่ำเสมอเพื่อช่วยลดไขมันในร่างกาย."
    elif input_data['systolic_bp'] > 140:
        exercise_recommendation = "การออกกำลังกาย: แนะนำให้ทำกิจกรรมที่ไม่หนักเกินไป เช่น ว่ายน้ำ หรือเดินเร็วเพื่อการควบคุมความดันโลหิต."
    else:
        exercise_recommendation = "การออกกำลังกาย: แนะนำให้ทำการออกกำลังกายที่หนักขึ้น เช่น วิ่ง หรือยกน้ำหนักเพื่อเสริมสร้างกล้ามเนื้อ."

    recommendations['exercise'] = exercise_recommendation

    # Food Recommendation
    if input_data['blood_sugar'] > 180:
        food_recommendation = "อาหาร: ควรหลีกเลี่ยงอาหารที่มีน้ำตาลสูงและเพิ่มอาหารที่มีดัชนีน้ำตาลต่ำ เช่น ข้าวกล้อง, ผักสด, ผลไม้ที่มีน้ำตาลต่ำ เช่น แอปเปิ้ล เพื่อควบคุมระดับน้ำตาล."
    elif input_data['blood_sugar'] > 120:
        food_recommendation = "อาหาร: ควรทานอาหารที่มีเส้นใยสูง เช่น ผัก, ถั่ว, ข้าวกล้อง, เพื่อควบคุมระดับน้ำตาลในเลือด."
    else:
        food_recommendation = "อาหาร: ควรเลือกทานอาหารที่มีโปรตีนสูง เช่น ปลา, ไก่, หรือเต้าหู้ และควรหลีกเลี่ยงอาหารไขมันสูง."

    recommendations['food'] = food_recommendation

    # Article Recommendation
    if input_data['mood'] == 'เศร้า':
        article_recommendation = "บทความ: แนะนำบทความเกี่ยวกับการจัดการอารมณ์และการลดความเครียด เช่น เทคนิคการหายใจ หรือการทำสมาธิเพื่อลดความเครียด."
    elif input_data['mood'] == 'หงุดหงิด':
        article_recommendation = "บทความ: แนะนำบทความเกี่ยวกับการผ่อนคลายและเทคนิคการจัดการอารมณ์ เช่น การทำกิจกรรมที่ช่วยให้รู้สึกผ่อนคลาย."
    else:
        article_recommendation = "บทความ: แนะนำบทความเกี่ยวกับการดูแลสุขภาพและการควบคุมเบาหวาน โดยการควบคุมอาหารและการออกกำลังกาย."

    recommendations['article'] = article_recommendation

    # คำแนะนำเพิ่มเติมหากมีความดันโลหิตสูง
    if input_data['systolic_bp'] > 140:
        blood_pressure_recommendation = "คำแนะนำเพิ่มเติม: ควรปรึกษาแพทย์เกี่ยวกับการควบคุมความดันโลหิต. การควบคุมความดันโลหิตสามารถช่วยลดความเสี่ยงจากโรคหลอดเลือดและหัวใจ."
        recommendations['blood_pressure'] = blood_pressure_recommendation
    
    return recommendations

# Sample Input Data
user_data = [
    {'BMI': 32, 'blood_sugar': 190, 'systolic_bp': 150, 'mood': 'เศร้า'},
    {'BMI': 28, 'blood_sugar': 140, 'systolic_bp': 130, 'mood': 'เฉยๆ'},
    {'BMI': 26, 'blood_sugar': 110, 'systolic_bp': 125, 'mood': 'ความสุข'},
    {'BMI': 35, 'blood_sugar': 200, 'systolic_bp': 160, 'mood': 'หงุดหงิด'},
    {'BMI': 30, 'blood_sugar': 180, 'systolic_bp': 145, 'mood': 'โกรธ'},
    {'BMI': 27, 'blood_sugar': 170, 'systolic_bp': 135, 'mood': 'ดี'},
    {'BMI': 40, 'blood_sugar': 220, 'systolic_bp': 170, 'mood': 'เศร้า'},
    {'BMI': 24, 'blood_sugar': 130, 'systolic_bp': 120, 'mood': 'เฉยๆ'},
    {'BMI': 29, 'blood_sugar': 155, 'systolic_bp': 125, 'mood': 'หงุดหงิด'},
    {'BMI': 33, 'blood_sugar': 195, 'systolic_bp': 155, 'mood': 'เศร้า'}
]

# สร้างตารางผลการทำนาย
results = []

# ทำนายผลและให้คำแนะนำ
for i, input_data in enumerate(user_data, 1):
    risk_percentage, health_score = predict_health(input_data)
    recommendations = health_recommendations(input_data)
    
    results.append({
        'User': i,
        'Health Score': health_score,
        'Risk Percentage (Diabetes)': f"{risk_percentage}%",
        'Exercise Recommendation': recommendations['exercise'],
        'Food Recommendation': recommendations['food'],
        'Article Recommendation': recommendations['article'],
        'Blood Pressure Recommendation': recommendations.get('blood_pressure', 'N/A')
    })

# สร้าง DataFrame
df_results = pd.DataFrame(results)

# แสดงผลในรูปแบบตาราง
print(df_results.to_string(index=False))


 User  Health Score Risk Percentage (Diabetes)                                                                                                       Exercise Recommendation                                                                                                                                      Food Recommendation                                                                                             Article Recommendation                                                                                                       Blood Pressure Recommendation
    1            40                         2% การออกกำลังกาย: แนะนำให้เดินเร็ว 30 นาทีต่อวัน หรือทำโยคะเพื่อเพิ่มความยืดหยุ่น. ควรทำอย่างสม่ำเสมอเพื่อช่วยลดไขมันในร่างกาย. อาหาร: ควรหลีกเลี่ยงอาหารที่มีน้ำตาลสูงและเพิ่มอาหารที่มีดัชนีน้ำตาลต่ำ เช่น ข้าวกล้อง, ผักสด, ผลไม้ที่มีน้ำตาลต่ำ เช่น แอปเปิ้ล เพื่อควบคุมระดับน้ำตาล. บทความ: แนะนำบทความเกี่ยวกับการจัดการอารมณ์และการลดความเครียด เช่น เทคนิคการหายใจ หรือการทำสมาธิเพื่อ

In [ ]:
#Step 11 Save the Model
import joblib

# Save the model
joblib.dump(best_model, 'health_model.pkl')

# Load the model (for future use)
loaded_model = joblib.load('health_model.pkl')
